<a href="https://colab.research.google.com/github/teyang-lau/coffee-joint-rec-sys/blob/main/WMF_LOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import io
username = 'tituslhy'
token = 'xxx'
github_session = requests.Session()
github_session.auth = (username,token)
train_url = 'https://raw.githubusercontent.com/teyang-lau/coffee-joint-rec-sys/main/data/processed/train_lol.csv'
val_url = 'https://raw.githubusercontent.com/teyang-lau/coffee-joint-rec-sys/main/data/processed/val_lol.csv'
test_url = 'https://raw.githubusercontent.com/teyang-lau/coffee-joint-rec-sys/main/data/processed/test_lol.csv'

## Get data

In [2]:
def get_data(url,username=username,token=token):
  download = github_session.get(url).content
  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
  df = df[['userid','shop','rating']]
  data = list(df.itertuples(index=False,name=None))
  return data

In [3]:
train = get_data(train_url)
train[0:5]

[('-5YMIME_WEin_by41Bj-3Q', 'liho-tea-singapore-117', 3.0),
 ('-5YMIME_WEin_by41Bj-3Q', 'old-hen-coffee-bar-singapore-2', 4.0),
 ('-5YMIME_WEin_by41Bj-3Q', 'two-men-bagel-house-singapore', 4.0),
 ('-5YMIME_WEin_by41Bj-3Q', 'old-airport-road-food-centre-singapore', 5.0),
 ('-G1YjYxjDpxOBzFgo36ORA', 'the-book-cafe-singapore', 3.0)]

In [4]:
val = get_data(val_url)
val[0:5]

[('-5YMIME_WEin_by41Bj-3Q', 'dutch-colony-coffee-singapore', 4.0),
 ('-G1YjYxjDpxOBzFgo36ORA', 'the-providore-singapore', 1.0),
 ('-WShM_YFbtG4OcE0vrFVyw', 'nylon-coffee-roasters-singapore', 4.0),
 ('-XaIf12ricWc5z5BRt9nnQ', 'tiong-bahru-bakery-singapore-4', 4.0),
 ('-fUWq6sOIEe1uTUhNKS9sQ', 'drips-singapore', 4.0)]

In [5]:
test = get_data(test_url)
test[0:5]

[('-5YMIME_WEin_by41Bj-3Q', 'luna-singapore', 3.0),
 ('-G1YjYxjDpxOBzFgo36ORA', 'common-man-coffee-roasters-singapore', 2.0),
 ('-WShM_YFbtG4OcE0vrFVyw', 'chye-seng-huat-hardware-singapore', 3.0),
 ('-XaIf12ricWc5z5BRt9nnQ', 'chye-seng-huat-hardware-singapore', 3.0),
 ('-fUWq6sOIEe1uTUhNKS9sQ', 'meng-kitchen-singapore', 5.0)]

In [6]:
print(len(train))
print(len(val))
print(len(test))

2822
497
497


## Get packages

In [7]:
!pip install --quiet cornac==1.14.2 adjustText

     |████████████████████████████████| 12.4 MB 5.5 MB/s 


In [8]:
import os
import sys
import itertools
import json

import scipy.sparse as sp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
%matplotlib inline

import cornac
from cornac.eval_methods import BaseMethod, CrossValidation
from cornac.models import WMF

%tensorflow_version 1.x
import tensorflow as tf

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")
print(f"Tensorflow version: {tf.__version__}")

SEED = 42
VERBOSE = True

TensorFlow 1.x selected.
System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Cornac version: 1.14.2
Tensorflow version: 1.15.2


In [9]:
base = BaseMethod.from_splits(train_data = train, 
                              test_data = val,
                              rating_threshold=3.5,
                              seed = SEED,
                              fmt='UIR')

eval_metrics = [
  cornac.metrics.NDCG(k=5),
  cornac.metrics.NCRR(k=5),
  cornac.metrics.Recall(k=5)
]

## Vanilla runs

In [10]:
K=50
lr = 0.001
iter = 1000
K2 = 100

#Default params:
# k=200, lambda_u=0.01, lambda_v=0.01, a=1, b=0.01, learning_rate=0.001, batch_size=128, max_iter=100

wmf = WMF(k=K, verbose=VERBOSE, seed=SEED, name=f"Vanilla WMF(K={K})")
wmf2 = WMF(k=K, learning_rate = lr, verbose=VERBOSE, seed=SEED, name=f"WMF(K={K}, lr = {lr})")
wmf3 = WMF(k=K, max_iter = iter, verbose=VERBOSE, seed=SEED, name=f"WMF(K={K}, num_iter = {iter})")
wmf4 = WMF(k=K, max_iter=iter, learning_rate = lr, verbose=VERBOSE, seed=SEED, name=f"WMF(K={K}, num_iter = {iter},lr = {lr})")
wmf5 = WMF(k=K2, verbose=VERBOSE, seed=SEED, name=f"Vanilla WMF(K={K2})")
wmf6 = WMF(k=K2, max_iter=iter, learning_rate = lr, verbose=VERBOSE, seed=SEED, name=f"WMF(K={K2}, num_iter = {iter},lr = {lr})")

exp = cornac.Experiment(eval_method=base, models=[wmf,wmf2,wmf3,wmf4,wmf5,wmf6], metrics=eval_metrics)
exp.run()

  0%|          | 0/100 [00:00<?, ?it/s]

Learning completed!


  0%|          | 0/100 [00:00<?, ?it/s]

Learning completed!


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Learning completed!


  0%|          | 0/100 [00:00<?, ?it/s]

Learning completed!


  0%|          | 0/1000 [00:00<?, ?it/s]

Learning completed!

TEST:
...
                                       | NCRR@5 | NDCG@5 | Recall@5 | Train (s) | Test (s)
-------------------------------------- + ------ + ------ + -------- + --------- + --------
Vanilla WMF(K=50)                      | 0.0036 | 0.0061 |   0.0141 |    7.2969 |   0.2205
WMF(K=50, lr = 0.001)                  | 0.0036 | 0.0061 |   0.0141 |    1.5904 |   0.2363
WMF(K=50, num_iter = 1000)             | 0.0056 | 0.0078 |   0.0141 |   13.7554 |   0.2194
WMF(K=50, num_iter = 1000,lr = 0.001)  | 0.0056 | 0.0078 |   0.0141 |   13.6702 |   0.2172
Vanilla WMF(K=100)                     | 0.0031 | 0.0044 |   0.0085 |    1.5867 |   0.2197
WMF(K=100, num_iter = 1000,lr = 0.001) | 0.0042 | 0.0066 |   0.0141 |   13.7127 |   0.2365



Increasing the number of iterations improves the score but not the learning rate. Increasing K might have resulted in some overfitting. Interestingly, decreasing the learning rate appeared to make no impact.

In [11]:
from scipy.stats import hmean

hmeans = []

for i in range(len(exp.result)):
  hmeans.append(float(hmean([exp.result[i].metric_avg_results['NCRR@5'],
                       exp.result[i].metric_avg_results['NDCG@5'],
                       exp.result[i].metric_avg_results['Recall@5']])))
  
print('Vanilla harmonic mean score: {:.4f}'.format(max(hmeans)))

Vanilla harmonic mean score: 0.0079


## Hyperparameter tuning

In [13]:
!pip install --quiet hyperopt

In [14]:
from hyperopt import tpe,hp,fmin,Trials, STATUS_OK

In [43]:
bestparams=[]
bestharmonic=0

NDCG=cornac.metrics.NDCG(5)
NCRR=cornac.metrics.NCRR(5)
Recall=cornac.metrics.Recall(5)

In [16]:
def ObjectiveF(params):
  global NDCG
  global NCRR
  global Recall
  global bestparams
  global bestharmonic

  latentk=params['latentk']
  learningrate = params['learningrate']
  lambda_u=params['lambda_u']
  lambda_v=params['lambda_v']
  b = params['b']
  iter = params['iter']

  wmf=WMF(k=latentk, 
          max_iter=iter, 
          a=1,
          b=b,
          learning_rate=learningrate,
          lambda_u=lambda_u, 
          lambda_v=lambda_v,
          seed=SEED)

  test_result, val_result = base.evaluate(model=wmf, 
                                          metrics=[NDCG,NCRR,Recall], 
                                          user_based=False, 
                                          show_validation=False)
  
  Sample_NCRR=test_result.metric_avg_results['NCRR@5']
  Sample_NDCG=test_result.metric_avg_results['NDCG@5']
  Sample_Recall=test_result.metric_avg_results['Recall@5']
  Sample_HM=hmean([Sample_NCRR,Sample_NDCG,Sample_Recall])
  loss=-Sample_HM
  reportstring=f"The Harmonic Mean for K={latentk}, lambda_u={lambda_u}, lambda_v = {lambda_v}, learningrate={learningrate}, b={b}, max_iter ={iter} is {loss}"
  print(reportstring)

  if Sample_HM>bestharmonic:
    bestparams=(latentk, learningrate, lambda_u, lambda_v,iter,b)
    bestharmonic=Sample_HM
  return loss

In [44]:
%%timeit
trials = Trials()

space = {
    'latentk': hp.choice('latentk', np.arange(50, 100, dtype=int)),
    'learningrate':hp.loguniform('learning_rate',-20,-15),
    'lambda_u': hp.loguniform('lambda_u', -15, -10),
    'lambda_v': hp.loguniform('lambda_v', -15, -10),
    'iter': hp.choice('iter', np.arange(200, 500, dtype=int)),
    'b' : hp.loguniform('b', -3, -1)
}

best=fmin(
    fn=ObjectiveF,
    space=space,
    algo=tpe.suggest,
    trials=trials,
    max_evals = 1
)

  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

  0%|          | 0/435 [00:00<?, ?it/s]

Learning completed!
The Harmonic Mean for K=87, lambda_u=5.301449574563928e-06, lambda_v = 9.13200947532363e-06, learningrate=2.9296871587404184e-09, b=0.25099810324060084, max_iter =435 is -0.0032701075580194583
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

  0%|          | 0/332 [00:00<?, ?it/s]

Learning completed!
The Harmonic Mean for K=62, lambda_u=3.890929618345889e-07, lambda_v = 2.179183444846076e-05, learningrate=2.1009736263018615e-07, b=0.05048838910568805, max_iter =332 is -0.008005009090342243
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

...

Learning completed!
The Harmonic Mean for K=60, lambda_u=4.3213086569527855e-06, lambda_v = 4.510206189030427e-05, learningrate=2.4318975701123634e-09, b=0.2572746517079985, max_iter =338 is -0.001154163782522384
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

  0%|          | 0/218 [00:00<?, ?it/s]

Learning completed!
The Harmonic Mean for K=65, lambda_u=5.860992654510805e-07, lambda_v = 1.17544058055698e-06, learningrate=4.178443367575775e-09, b=0.08349084633362631, max_iter =218 is -0.0046854040061089005
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

  0%|          | 0/231 [00:00<?, ?it/s]

Learning completed!
The Harmonic Mean for K=83, lambda_u=2.137710979144415e-05, lambda_v = 1.0016924893074219e-05, learningrate=3.614730254104161e-08, b=0.12960464107339542, max_iter =231 is -0.013212538127304099
100%|██████████| 1/1 [00:03<00:00,  3.54s/it, best loss: -0.013212538127304099]
1 loop, best of 5: 3.51 s per loop


In [45]:
print(f'The best hyperparameters are: K={bestparams[0]},lr = {bestparams[1]},lambda_u = {bestparams[2]}, lambda_v = {bestparams[3]}, max_iter = {bestparams[4]}, b = {bestparams[5]}')
print(f'The harmonic mean from the best parameters is: {bestharmonic}')

The best hyperparameters are: K=83,lr = 3.614730254104161e-08,lambda_u = 2.137710979144415e-05, lambda_v = 1.0016924893074219e-05, max_iter = 231, b = 0.12960464107339542
The harmonic mean from the best parameters is: 0.013212538127304099


Testing model against test set to ensure that the model is generalizable.

In [46]:
base2 = BaseMethod.from_splits(train_data = train, 
                              test_data = test,
                              rating_threshold=3.5,
                              seed = SEED,
                              fmt='UIR')

wmf_f = WMF(k=bestparams[0], 
            learning_rate = bestparams[1], 
            lambda_u = bestparams[2], 
            lambda_v = bestparams[3],
            max_iter = bestparams[4],
            b = bestparams[5],
            verbose=VERBOSE, seed=SEED, 
            name="WMF(K={}, lr = {:.04g}, lambda_u = {:.04g},lambda_v ={:.04g}, max_iter = {}, b = {:.04g})".format(bestparams[0],bestparams[1],bestparams[2],bestparams[3],bestparams[4],bestparams[5]))

exp = cornac.Experiment(eval_method=base, models=[wmf_f], metrics=eval_metrics)
exp.run()

  0%|          | 0/231 [00:00<?, ?it/s]

Learning completed!

TEST:
...
                                                                                                | NCRR@5 | NDCG@5 | Recall@5 | Train (s) | Test (s)
----------------------------------------------------------------------------------------------- + ------ + ------ + -------- + --------- + --------
WMF(K=83, lr = 3.615e-08, lambda_u = 2.138e-05,lambda_v =1.002e-05, max_iter = 231, b = 0.1296) | 0.0096 | 0.0128 |   0.0225 |    3.4272 |   0.2281



In [47]:
print('Test harmonic mean score: {:.4f}'.format(float(hmean([exp.result[0].metric_avg_results['NCRR@5'],
                                                             exp.result[0].metric_avg_results['NDCG@5'],
                                                             exp.result[0].metric_avg_results['Recall@5']]))))

Test harmonic mean score: 0.0132


The harmonic mean score is sufficiently close to the training score. Model is sufficiently generalizable.